In [1]:
import os
import numpy as np
import efficientnet.tfkeras
from tensorflow.keras.models import load_model
from tensorflow.keras import models
import tensorflow as tf
from focal_loss import BinaryFocalLoss

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [3]:
model_dir ='/media/tohn/SSD/ModelTrainByImages/R2_2/models/B5R2b5_AN_1FC_fold2_3.h5'
model = load_model(model_dir)
height = width = model.input_shape[1]

x = model.get_layer('head_pooling').output
prediction_layer = model.output
model2 = models.Model(inputs= model.input, outputs=[prediction_layer,x]) 

In [5]:
#ไม่ต้องเปลี่ยนก็ได้ เพราะเอา Label มาใช้เฉยๆ 
# validation
import pandas as pd
base_dir  = '/media/tohn/SSD/Images/Image2/validation/'
dataframe = pd.read_csv( '/home/yupaporn/codes/USAI/Validationdf_fold1_3.csv')
validation_dir = os.path.join(base_dir, 'validation')

#Train
train_df = pd.read_csv( '/home/yupaporn/codes/USAI/Traindf_fold1_3.csv')
base_dir0 = '/media/tohn/SSD/Images/Image1/'
os.chdir(base_dir0)
train_dir = os.path.join(base_dir0, 'train')

In [6]:
import pandas as pd
df0 = pd.read_csv (r'/home/yupaporn/codes/USAI/Validationdf_fold2_3.csv')
print(df0 .shape)
dataframe = df0[(df0['Path Crop']!='None' )&(df0['Path Crop']!='Nan')]
# a = df01[df01['Case'].between(1, 10)]
# dataframe = df01[df01['Abs Position']!='P8']
print(dataframe.shape)
print('Normal: ',dataframe[dataframe['Class']=='Normal'].shape)
print('Abnormal: ',dataframe[dataframe['Class']=='Abnormal'].shape)
dataframe.head(5)

(1032, 22)
(1032, 22)
Normal:  (559, 22)
Abnormal:  (473, 22)


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Case,Abs Position,Sub Position,Class,Sub_class,Path Full,Path Crop,...,tagName,originalImage,left,top,width,height,Rleft,Rtop,Rwidth,Rheight
0,0,668,668,8,P1,P1,Abnormal,AB01,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,MildFattyLiver,https://irisprodseatraining.blob.core.windows....,130.0,133.0,614.0,476.0,0.131769,0.148873,0.619134,0.529880
1,1,672,672,8,P1,P1,Abnormal,AB02,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,ModerateFattyLiver,https://irisprodseatraining.blob.core.windows....,272.0,105.0,531.0,641.0,0.270496,0.117298,0.526978,0.713131
2,2,678,678,8,P1,P1,Abnormal,AB03,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,SevereFattyLiver,https://irisprodseatraining.blob.core.windows....,207.0,103.0,531.0,715.0,0.215030,0.115278,0.550752,0.795960
3,3,681,681,8,P1,P1,Abnormal,AB04,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,Cirrhosis,https://irisprodseatraining.blob.core.windows....,238.0,60.0,452.0,360.0,0.247450,0.066793,0.468927,0.400000
4,4,685,685,8,P1,P1,Abnormal,AB05,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,/media/tohn/HDD/VISION_dataset/USAI/ABnormal01...,...,PDF1,https://irisprodseatraining.blob.core.windows....,304.0,98.0,483.0,449.0,0.303804,0.109217,0.482759,0.498990


In [6]:
# dataframe.info()

In [7]:
batch_size = 64
epochs = 10

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      brightness_range=[0.5,1.5],
      shear_range=0.4,
      zoom_range=0.2,
      horizontal_flip=False,
      fill_mode='nearest')

train_generator = train_datagen.flow_from_dataframe(
        dataframe = dataframe,
        directory = train_dir,
        x_col = 'Path Crop',
        y_col = 'Class',
        target_size = (height, width),
        batch_size=batch_size,
        color_mode= 'rgb',
        class_mode='categorical')

#label
labels = (train_generator.class_indices)
labels = dict((v,k.replace("C","")) for k,v in labels.items())
print(labels)

Found 1032 validated image filenames belonging to 2 classes.
{0: 'Abnormal', 1: 'Normal'}


In [8]:
from tensorflow.keras.preprocessing import image
def predict_image(img_path):
    # Read the image and resize it
    img = image.load_img(img_path, target_size=(height, width))
    # Convert it to a Numpy array with target shape.
    x = image.img_to_array(img)
    # Reshape
    x = x.reshape((1,) + x.shape)
    x /= 255.
    result = model2.predict([x])
    result1 = result[0][0]
    result2 = result[1][0]

    return result1, result2

#Predict
pred_list = list()
prob_list = list()
vectorlist = list()
img_path=dataframe['Path Crop'].tolist()
for i in range(0,len(img_path)):
    result1,result2 = predict_image(img_path[i])
    result = np.argmax(result1)
    pred_list.append(labels[result])
    prob_list.append(result1[result])
    vectorlist.append(result2.tolist())
    
dataframe['category'] = pred_list
dataframe['Prob'] = prob_list

In [9]:
vector_array = np.array(vectorlist)
vector_array

array([[ 0.4461658 , -0.1458414 ,  0.63796562, ...,  0.16417041,
         0.26954222, -0.14477883],
       [ 0.17227867, -0.12442519,  0.36219427, ...,  0.15380749,
        -0.08486952, -0.0386778 ],
       [ 0.1736192 , -0.09796079,  0.51654291, ...,  0.52762473,
         0.02890034, -0.15258193],
       ...,
       [-0.09790668,  0.37151122, -0.18950583, ..., -0.18770887,
        -0.20376872,  0.46229878],
       [-0.18200767,  0.0033174 , -0.20811471, ..., -0.16739728,
        -0.18511635,  0.14064269],
       [-0.18200767,  0.0033174 , -0.20811471, ..., -0.16739728,
        -0.18511635,  0.14064269]])

In [10]:
dataset = pd.DataFrame(vector_array[:, :])
dataset

,0,1,2,3,4,5,6,7,8,9,...,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047
0,0.446166,-0.145841,0.637966,0.648146,-0.231161,0.296943,0.598006,-0.228232,-0.180162,0.370153,...,-0.196645,0.210500,-0.198223,-0.173900,-0.190615,0.357889,0.506763,0.164170,0.269542,-0.144779
1,0.172279,-0.124425,0.362194,0.179361,-0.127346,-0.009097,0.080945,-0.155355,-0.200693,-0.014353,...,-0.170056,-0.018566,-0.211472,-0.112752,-0.105687,-0.154341,0.053849,0.153807,-0.084870,-0.038678
2,0.173619,-0.097961,0.516543,0.357135,-0.218639,0.190848,0.244322,-0.129721,-0.180350,0.011070,...,-0.165540,0.234209,-0.117046,-0.165374,-0.150880,-0.066432,0.327964,0.527625,0.028900,-0.152582
3,-0.015411,-0.163745,0.162167,0.028496,-0.220971,-0.004650,0.171672,-0.157929,-0.216944,-0.112352,...,-0.156527,-0.107296,-0.208583,-0.192354,-0.114738,0.068804,0.001964,-0.089300,-0.159361,-0.120028
4,-0.108861,0.010125,-0.029730,-0.044152,0.027052,0.001503,0.106726,-0.138251,-0.112890,-0.185297,...,-0.120531,-0.006782,-0.157785,0.023598,-0.182867,-0.114160,0.082085,0.074840,0.070101,-0.052123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027,-0.158955,0.262772,-0.174435,-0.005281,0.685930,-0.093771,-0.148601,0.294996,0.270731,-0.185894,...,0.657653,-0.195917,0.350960,0.370807,0.545062,-0.139943,-0.183224,-0.124785,-0.172539,0.294996
1028,-0.141295,0.488426,-0.075364,-0.154368,0.669906,-0.098334,0.054864,0.757183,0.418839,-0.217763,...,0.325305,-0.131143,0.386070,-0.024301,0.527810,-0.115136,-0.094787,-0.095182,-0.199221,0.219805
1029,-0.097907,0.371511,-0.189506,0.005109,0.324590,-0.221479,-0.193559,0.114645,-0.001883,-0.224781,...,0.426198,-0.211891,-0.011296,0.198828,0.571136,-0.222612,-0.193280,-0.187709,-0.203769,0.462299
1030,-0.182008,0.003317,-0.208115,-0.117425,0.519947,-0.194665,-0.194366,-0.027825,-0.030836,-0.201721,...,0.355306,-0.182776,0.227576,-0.054771,0.297868,-0.177855,-0.184304,-0.167397,-0.185116,0.140643


In [11]:
a = dataframe['Class'].tolist()
b = dataframe['Sub Position'].tolist()
c = dataframe['Views'].tolist()
d = dataframe['Sub_class'].tolist()
dataset['Class'] = a 
dataset['SubPosition'] = b
dataset['Views'] = c
dataset['Sub_class'] = d

In [12]:
dataset

,0,1,2,3,4,5,6,7,8,9,...,2042,2043,2044,2045,2046,2047,Class,SubPosition,Views,Sub_class
0,0.446166,-0.145841,0.637966,0.648146,-0.231161,0.296943,0.598006,-0.228232,-0.180162,0.370153,...,-0.190615,0.357889,0.506763,0.164170,0.269542,-0.144779,Abnormal,P1,FP-A,AB01
1,0.172279,-0.124425,0.362194,0.179361,-0.127346,-0.009097,0.080945,-0.155355,-0.200693,-0.014353,...,-0.105687,-0.154341,0.053849,0.153807,-0.084870,-0.038678,Abnormal,P1,FP-A,AB02
2,0.173619,-0.097961,0.516543,0.357135,-0.218639,0.190848,0.244322,-0.129721,-0.180350,0.011070,...,-0.150880,-0.066432,0.327964,0.527625,0.028900,-0.152582,Abnormal,P1,FP-A,AB03
3,-0.015411,-0.163745,0.162167,0.028496,-0.220971,-0.004650,0.171672,-0.157929,-0.216944,-0.112352,...,-0.114738,0.068804,0.001964,-0.089300,-0.159361,-0.120028,Abnormal,P1,FP-A,AB04
4,-0.108861,0.010125,-0.029730,-0.044152,0.027052,0.001503,0.106726,-0.138251,-0.112890,-0.185297,...,-0.182867,-0.114160,0.082085,0.074840,0.070101,-0.052123,Abnormal,P1,FP-A,AB05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1027,-0.158955,0.262772,-0.174435,-0.005281,0.685930,-0.093771,-0.148601,0.294996,0.270731,-0.185894,...,0.545062,-0.139943,-0.183224,-0.124785,-0.172539,0.294996,Normal,P8,FP-E,Normal
1028,-0.141295,0.488426,-0.075364,-0.154368,0.669906,-0.098334,0.054864,0.757183,0.418839,-0.217763,...,0.527810,-0.115136,-0.094787,-0.095182,-0.199221,0.219805,Normal,P8,FP-E,Normal
1029,-0.097907,0.371511,-0.189506,0.005109,0.324590,-0.221479,-0.193559,0.114645,-0.001883,-0.224781,...,0.571136,-0.222612,-0.193280,-0.187709,-0.203769,0.462299,Normal,P8,FP-E,Normal
1030,-0.182008,0.003317,-0.208115,-0.117425,0.519947,-0.194665,-0.194366,-0.027825,-0.030836,-0.201721,...,0.297868,-0.177855,-0.184304,-0.167397,-0.185116,0.140643,Normal,P8,FP-E,Normal


In [13]:
os.chdir( '/home/yupaporn/codes/USAI/')
testdf = dataset
testdf.to_csv('FVvalidation_AN_RF_AN_fold2_3.csv')